# Food security portal datasets

Running this notebooks collects all data sources from http://www.foodsecurityportal.org/ maintained by the EU. Available datasets are automatically scrapped from homepage http://www.foodsecurityportal.org/api/about.

## Settings

In [6]:
# helpers
from pathlib import Path
import urllib.request
from datetime import datetime
import shutil
import tqdm

# data processing
import pandas as pd
from bs4 import BeautifulSoup

In [18]:
def scarp_all_datasets_from_foodSecurityPortal(
    url="http://www.foodsecurityportal.org/api/about",
):
    # Return a Python list that contains all links pointing to datasets to date

    with urllib.request.urlopen(url) as page:
        html = BeautifulSoup(page.read().decode("utf-8"), "html.parser")
        all_links_in_page = html.find_all("a")
        all_not_empty_links = [
            link.get("href")
            for link in all_links_in_page
            if (not link.get("href") is None)
        ]
        dataset_links = [
            link
            for link in all_not_empty_links
            if (
                (link != "/api")
                and (not "documentation" in link)
                and (not "about" in link)
                and ("api" in link)
            )
        ]
        return dataset_links


def get_csv_file_from_dataset(
    dataset_links,
    url="http://www.foodsecurityportal.org",
    output_path=Path("../../data/clean/food_security/"),
):
    # Simply download corresponding .csv file while removing existing ones
    # Could not think of better solution since it's hard to check for updates...

    for dataset_url in dataset_links:
        filename = dataset_url.split("/")[-1] + ".csv"
        urllib.request.urlretrieve(url + dataset_url + ".csv", output_path / filename)

## Get all datasets from http://www.foodsecurityportal.org/

In [ ]:
# Get the datasets urls
dataset_links = scarp_all_datasets_from_foodSecurityPortal()
# Download the csv, save it
get_csv_file_from_dataset(dataset_links)